# Cat & dog image classification

Выполнил: студент группы А-14м-16 Мигаль Иван.

Описание:

Решение задачи классификации образов (кошек и собак) с помощью keras. 

Для начала подключим необходимые модули и выведем их версии, в том числе версию python3. 
Также выведем абсолютный путь к папке с python3 (для разработчиков).

In [1]:
%matplotlib inline
import keras
import os
import matplotlib
import urllib.request
from tqdm import tqdm_notebook
import cv2
from matplotlib import pyplot as plt
import numpy as np
import sys
import getpass
import zipfile
import glob

usrname = getpass.getuser()

print('Версия python3:', sys.version)
print('Версия keras:', keras.__version__)
print('Версия matplotlib:', matplotlib.__version__)
print('Версия urlib.request', urllib.request.__version__)
print('Версия numpy', np.__version__)
print('Версия OpenCV:', cv2.__version__)
print('Абсолютный путь к папке python3:', sys.executable)
print('Имя пользователя', usrname)

Using TensorFlow backend.


Версия python3: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Версия keras: 2.0.9
Версия matplotlib: 1.5.3
Версия urlib.request 3.5
Версия numpy 1.11.1
Версия OpenCV: 3.2.0
Абсолютный путь к папке python3: /home/ivmig/anaconda3/bin/python
Имя пользователя ivmig


Теперь скачаем данные. Это размеченные картинки кошек и собак с соревнования Kaggle. Получить доступ к ним с одноименного сайта проблематично. К счастью, на сайте Майкрософт эти данные тоже есть, поэтому скачивать будем оттуда.

In [2]:
%%time

# Создаем директорию, если таковая отсутствует
path_to_data_no_split = os.getcwd() + '/data_no_split/'
if not os.path.exists(path_to_data_no_split):
    os.makedirs(path_to_data_no_split)

# Скачивание
url = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'
path_to_zip_file = path_to_data_no_split + 'cat_dog_images.zip'
if not os.path.exists(path_to_zip_file):
    urllib.request.urlretrieve(url, path_to_zip_file)  

# Распаковка архива
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(path_to_data_no_split)
zip_ref.close()

CPU times: user 3.18 s, sys: 8.43 s, total: 11.6 s
Wall time: 39.5 s


In [ ]:
# Размер изображения, которое подается на вход сети
img_width, img_height = 250, 250

# Классы
classes = ["dog", "cat"]
num_classes = len(classes)

# Загрузить изображение
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height), cv2.INTER_LINEAR)
    return resized

# Загрузить данные
def load_data():
    X_data = []
    X_data_id = []
    y_data = []

    print('Loading images...')
    folders = ["Dog", "Cat"]
    
    for fld in folders:
        index = folders.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(path_to_data_no_split+'PetImages/', fld, '*g')
        files = glob.glob(path)
        
        for fl in tqdm_notebook(files):
            flbase = os.path.basename(fl)
            # Битые картинки не используем
            try:
                img = load_images(fl)
            except:
                continue
            X_data.append(img)
            X_data_id.append(flbase)
            y_data.append(index)
            
    return X_data, y_data, X_data_id

# Нормализовать изображения (перевод цвета из [0, 255] в [0, 1])
def normalize_data():
    data_data, data_target, data_id = load_data()

    data_data = np.array(data_data, dtype=np.uint8)
    data_target = np.array(data_target, dtype=np.uint8)

    data_data = data_data.transpose((0, 3, 1, 2))

    data_data = data_data.astype('float32')
    data_data = data_data / 255
    data_target = np_utils.to_categorical(data_target, num_classes)

    print('Shape of data:', data_data.shape)
    return data_data, data_target, data_id


In [ ]:
%%time

data_data, data_target, data_id = normalize_data()

Loading images...
Loading Dog files (Index: 0)


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
